In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
!pip install tensorflow[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 76.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are

In [3]:
!pip install tensorflow_addons

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa

import time

In [5]:
tf.random.set_seed(51)

In [6]:
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [7]:
X_train_full = np.concatenate((X_train_raw, X_test_raw))
y_train_full = np.concatenate((y_train_raw, y_test_raw))

In [8]:
valid = pd.read_csv('../input/digit-recognizer/train.csv').to_numpy()
test = pd.read_csv('../input/digit-recognizer/test.csv').to_numpy()
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv').to_numpy()

In [9]:
train_Y = keras.utils.to_categorical(y_train_full)
valid_Y = keras.utils.to_categorical(valid[:,0])

# Preprocess Normalisation :
train_X = X_train_full / 255.0
valid_X = valid[:,1:] / 255.0
test_X = test / 255.0

# Preprocess for tensor inputs :
train_X = train_X.reshape(-1, 28, 28, 1)
valid_X = valid_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

# Verification of shapes :
print(train_X.shape,valid_X.shape,test_X.shape)

(70000, 28, 28, 1) (42000, 28, 28, 1) (28000, 28, 28, 1)


In [10]:
batch_size = 128
epochs = 25
steps = len(train_X) // batch_size
labels = 10

In [11]:
train_DS = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
valid_DS = tf.data.Dataset.from_tensor_slices((valid_X, valid_Y))

train_DS = train_DS.shuffle(steps * batch_size).batch(batch_size).repeat().prefetch(tf.data.AUTOTUNE)
valid_DS = valid_DS.batch(batch_size)

2022-10-30 16:05:12.956196: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
class Model:
    
    @staticmethod
    
    def build(height, width, depth):
        model = keras.Sequential()
        
        # Input
        model.add(Input((height, width, depth)))
        
        # CNN LAYERS
        model.add(Conv2D(32, (3, 3), activation = 'relu'))
        model.add(Conv2D(32, (3, 3), activation = 'relu'))
        model.add(MaxPooling2D(2, 2))
        model.add(BatchNormalization())
        
        model.add(Conv2D(64, (3, 3), activation = 'relu'))
        model.add(Conv2D(64, (3, 3), activation = 'relu'))
        model.add(MaxPooling2D(2, 2))
        model.add(BatchNormalization())
        
        model.add(Conv2D(128, (3, 3), activation = 'relu'))
        model.add(MaxPooling2D(2, 2))
        model.add(BatchNormalization())

        model.add(Flatten())
        
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(10, activation = 'softmax'))
        
        return model

In [13]:
model = Model.build(28,28,1)
model.summary()

optimizer = keras.optimizers.SGD(learning_rate = 5e-4)

early_stopping = keras.callbacks.EarlyStopping(patience = 5,
                                              min_delta = 1e-3,
                                              restore_best_weights = True)

chkpt = keras.callbacks.ModelCheckpoint(filepath = 'best_model_todate', save_best_only = True, save_weights_only = True)

endNan = keras.callbacks.TerminateOnNaN()

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

LRS = keras.callbacks.LearningRateScheduler(scheduler)

LRP = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 5, verbose = 3)

model.compile(
    optimizer = 'SGD',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
    steps_per_execution = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 12, 12, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0

In [14]:
%%time

epochs = 40 # Model was trained on 20 epoch before the last 5 who are here

Mdl1 = model.fit(train_DS,
                validation_data = valid_DS,
                epochs = EPOCHS,
                steps_per_epoch = steps,
                callbacks = [early_stopping, LRS, chkpt, LRP, endNan])

NameError: name 'EPOCHS' is not defined

In [15]:
df = pd.DataFrame(Mdl1.history)
df.index +=1

fig = plt.figure(figsize = (20, 5))
fig.suptitle('Validation metrics')

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.set_xticks(range(len(df)+1))
ax2.set_xticks(range(len(df)+1))

df[['loss', 'val_loss']].plot(title = "Cross Entropy", grid = True, ax = ax1)
df[['accuracy', 'val_accuracy']].plot(title = "Accuracy", grid = True, ax = ax2)

plt.show()

NameError: name 'Mdl1' is not defined

In [ ]:
model.save('model.h5', save_format = 'h5')

In [ ]:
pred = model.predict(test_X)
classes = pred.argmax(axis=-1)


sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sub.Label = classes
sub.head()

In [ ]:
sub.to_csv('submission_3.csv', index = False)